---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def acronyms(raw):
    for acronym, name in states.items():
        if name == raw:
            return acronym


def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data=[]
    with open('university_towns.txt','r') as f:
        file=f.read().split('\n')
    
    
    for element in file:
        if '[edit]' in element:
            state=element
        else:
            data.append([state,element])                
            

    data=pd.DataFrame(data,columns=["State", "RegionName"])
    data['State']=data['State'].str.split('[').str[0]
    data['RegionName']=data['RegionName'].str.split(' ()').str[0]

    data['State']=data['State'].apply(acronyms)

    
    return data

get_list_of_university_towns()

,State,RegionName
0,AL,Auburn
1,AL,Florence
2,AL,Jacksonville
3,AL,Livingston
4,AL,Montevallo
5,AL,Troy
6,AL,Tuscaloosa
7,AL,Tuskegee
8,AK,Fairbanks
9,AZ,Flagstaff


In [4]:
gdp=pd.read_excel('gdplev.xls')
gdp=gdp.loc[7:,['Unnamed: 4','Unnamed: 5','Unnamed: 6']].rename(columns={'Unnamed: 4':'Quarters',
                                                                     'Unnamed: 5':'GDP in billions of current dollars',
                                                                     'Unnamed: 6':'GDP in billions of chained 2009 dollars'})
    
gdp=gdp.reset_index(drop=True).drop(index=[raw for raw in range(0,(2000-1947)*4)])

def get_recession_start():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''    

    for i in range(1,len(gdp)-3):
        if (gdp.iloc[i,2]<gdp.iloc[i-1,2]) and (gdp.iloc[i+1,2]<gdp.iloc[i,2])\
        and (gdp.iloc[i+2,2]>gdp.iloc[i+1,2]) and (gdp.iloc[i+3,2]>gdp.iloc[i+2,2]):
            return gdp.iloc[i,0]

#get_recession_start()

In [5]:
def get_recession_end():
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    for i in range(1,len(gdp)-3):
        if (gdp.iloc[i,2]<gdp.iloc[i-1,2]) and (gdp.iloc[i+1,2]<gdp.iloc[i,2])\
        and (gdp.iloc[i+2,2]>gdp.iloc[i+1,2]) and (gdp.iloc[i+3,2]>gdp.iloc[i+2,2]):
            return gdp.iloc[i+3,0]
    
       
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    for i in range(1,len(gdp)-3):
        if (gdp.iloc[i,2]<gdp.iloc[i-1,2]) and (gdp.iloc[i+1,2]<gdp.iloc[i,2])\
        and (gdp.iloc[i+2,2]>gdp.iloc[i+1,2]) and (gdp.iloc[i+3,2]>gdp.iloc[i+2,2]):
            if gdp.iloc[i,2]<gdp.iloc[i+1,2]:
                return gdp.iloc[i,0]
            else:
                return gdp.iloc[i+1,0]
                     
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    data=pd.read_csv('City_Zhvi_AllHomes.csv')
    data.drop(columns=data.loc[:,'1996-04':'1999-12'].columns,inplace=True)
    i=6
    for year in range(2000,2017):
        for quarter in range(1,5):
            data[str(year)+'q'+str(quarter)]=data[data.columns[i:i+3]].apply(np.mean,axis=1)
            i=i+3
    
    data.drop(columns='2016q4',inplace=True)
    
    return data.drop(columns=data.loc[:,'2000-01':'2016-08'].columns).set_index(['State','RegionName'])

convert_housing_data_to_quarters()


,,RegionID,Metro,CountyName,SizeRank,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,6181,New York,Queens,1,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,5.872000e+05
CA,Los Angeles,12447,Los Angeles-Long Beach-Anaheim,Los Angeles,2,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,4.583889e+05
IL,Chicago,17426,Chicago,Cook,3,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,1.874667e+05
PA,Philadelphia,13271,Philadelphia,Philadelphia,4,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,1.034667e+05
AZ,Phoenix,40326,Phoenix,Maricopa,5,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,1.674111e+05
NV,Las Vegas,18959,Las Vegas,Clark,6,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,1.775000e+05
CA,San Diego,54296,San Diego,San Diego,7,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,4.341333e+05
TX,Dallas,38128,Dallas-Fort Worth,Dallas,8,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,1.276889e+05
CA,San Jose,33839,San Jose,Santa Clara,9,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,6.728889e+05


In [8]:
def run_ttest():
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # new data showing the decline or growth of housing prices between the recession start and the recession bottom
    recession_house_pricing=convert_housing_data_to_quarters()[['2009q1','2009q2']]
    recession_house_pricing['-decline/+growth']=recession_house_pricing['2009q2']-recession_house_pricing['2009q1']
    
    # adding a new column to distinguish university towns
    list_of_university_towns=get_list_of_university_towns().copy()
    list_of_university_towns['university_towns']=True
    
    # merging 'recession_house_pricing' and 'list_of_university_towns' 
    data=pd.merge(recession_house_pricing,list_of_university_towns,how='left',right_on=['RegionName','State'],left_on=['RegionName','State'])
    # distinguish non university towns
    data['university_towns']=data['university_towns'].fillna(False)
    # dropping null rows (rows with no quarters information)
    data=data.dropna()
    # segmenting data
    university_towns=data.loc[data['university_towns']==True]
    non_university_towns=data.loc[data['university_towns']==False]
    # calculating the p_value
    p_value=ttest_ind(university_towns['-decline/+growth'], non_university_towns['-decline/+growth'])[1]
    # calculating the mean price ratio
    if university_towns['-decline/+growth'].mean()/len(university_towns) < non_university_towns['-decline/+growth'].mean()/len(non_university_towns):
        better= "university town"
    else:
        better="non-university town"
            
    
    return p_value<0.01,p_value,better

run_ttest()



(True, 0.003427520745617828, 'university town')